In [1]:
import numpy as np
import glob
import os
import torch
import json
import torch.nn as nn
import torch.nn.functional as F


from torch.optim import Adam

from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

# Dataset Class

In [2]:
class Debris_State_Series_Dataset(Dataset):
    def __init__(self, root_dir, series_length, array_size=128, apply_scaling=False):
        self.array_size = array_size
        self.series_length = series_length
        self.data_series = []
        self.model_ids = []  # List to store the model IDs
        self.terrain_files = {}
        self.apply_scaling = apply_scaling

        def get_state_number(file_path):
            return int(file_path.split('_')[-1].split('.')[0])

        model_dirs = glob.glob(os.path.join(root_dir, '*'))
        
        for model_dir in model_dirs:
            if os.path.isdir(model_dir):
                # Extract the model ID from the model_dir path
                model_id = os.path.basename(model_dir)

                file_patterns = {
                    'velocity': os.path.join(model_dir, f'04_FinalProcessedData_{str(self.array_size)}', 'velocity', '*_velocity_*.npy'),
                    'thickness': os.path.join(model_dir, f'04_FinalProcessedData_{str(self.array_size)}', 'thickness', '*_thickness_*.npy')
                }
                file_collections = {key: sorted(glob.glob(pattern), key=get_state_number) for key, pattern in file_patterns.items()}

                terrain_pattern = os.path.join(model_dir, f'04_FinalProcessedData_{str(self.array_size)}', 'elevation', '*_elevation.npy')
                terrain_file = glob.glob(terrain_pattern)

                if terrain_file:
                    self.terrain_files[model_dir] = terrain_file[0]

                for i in range(len(file_collections['velocity']) - self.series_length):
                    velocity_series = file_collections['velocity'][i:i+self.series_length+1]
                    thickness_series = file_collections['thickness'][i:i+self.series_length+1]
                    self.data_series.append((velocity_series, thickness_series))
                    self.model_ids.append(model_id)

    def compute_scaling_factors(self):
        # Ensure that scaling is intended before computing factors
        if not self.apply_scaling:
            raise RuntimeError("Scaling factors called to be computed when scaling is not applied.")

        # Initialize min and max values with infinities
        self.min_elevation = np.inf
        self.max_elevation = -np.inf
        self.min_velocity = np.inf
        self.max_velocity = -np.inf
        self.min_thickness = np.inf
        self.max_thickness = -np.inf

        # Compute min and max values over the training set
        for model_dir, terrain_path in self.terrain_files.items():
            terrain = np.load(terrain_path)
            self.min_elevation = min(self.min_elevation, terrain.min())
            self.max_elevation = max(self.max_elevation, terrain.max())

            # Assuming that velocity and thickness files are matched in the data_series
            for velocity_series, thickness_series in self.data_series:
                if model_dir in velocity_series[0]:  # Checking if the series belongs to the current model directory
                    for velocity_path, thickness_path in zip(velocity_series, thickness_series):
                        velocity = np.load(velocity_path)
                        thickness = np.load(thickness_path)
                        self.min_velocity = min(self.min_velocity, velocity.min())
                        self.max_velocity = max(self.max_velocity, velocity.max())
                        self.min_thickness = min(self.min_thickness, thickness.min())
                        self.max_thickness = max(self.max_thickness, thickness.max())

    def scale_data(self, terrain, velocity, thickness):
        # Apply Min-Max scaling to each feature
        terrain_scaled = (terrain - self.min_elevation) / (self.max_elevation - self.min_elevation) * 10
        velocity_scaled = (velocity - self.min_velocity) / (self.max_velocity - self.min_velocity) * 10 
        thickness_scaled = (thickness - self.min_thickness) / (self.max_thickness - self.min_thickness) * 10
        return terrain_scaled, velocity_scaled, thickness_scaled


    def __len__(self):
        return len(self.data_series)

    def __getitem__(self, idx):
        velocity_series, thickness_series = self.data_series[idx]
        
        # Load velocity and thickness data for each state in the series
        velocity_data = [np.load(path) for path in velocity_series]
        thickness_data = [np.load(path) for path in thickness_series]

        # Get the model directory from the velocity path
        model_dir = os.path.dirname(os.path.dirname(os.path.dirname(velocity_series[0])))

        # Load the corresponding terrain file
        terrain_path = self.terrain_files[model_dir]
        terrain = np.load(terrain_path)

        if self.apply_scaling:
            # Apply scaling to the loaded data
            scaled_data = [self.scale_data(terrain, v, t) for v, t in zip(velocity_data, thickness_data)]
            terrain_scaled = [data[0] for data in scaled_data]
            velocity_scaled = [data[1] for data in scaled_data]
            thickness_scaled = [data[2] for data in scaled_data]

        else:
            # No scaling applied, use the original values
            terrain_scaled, velocity_scaled, thickness_scaled = terrain, velocity_data, thickness_data

        # Stack arrays as channels for CNN input and output
        current_seq = [np.stack((ts, v, t), axis=0) for ts, v, t in zip(terrain_scaled[:-1], velocity_scaled[:-1], thickness_scaled[:-1])]
        next_seq = np.stack((velocity_scaled[-1], thickness_scaled[-1]), axis=0)

        # Add a sequence length dimension of 1 to next_seq
        next_seq = next_seq[np.newaxis, ...]

        # print(f"Current sequence shape: {np.array(current_seq).shape}")
        # print(f"Next sequence shape: {next_seq.shape}")

        return torch.from_numpy(np.array(current_seq)).float(), torch.from_numpy(next_seq).float()

    def create_dataloaders(self, split_proportions, batch_size, random_state=42):
        # Unpack the proportions for clarity
        train_proportion, val_proportion, test_proportion = split_proportions
        
        # Assert that the proportions sum to 1
        assert np.isclose(sum(split_proportions), 1.0), "Proportions must sum up to 1."
        
        # Create a list of unique model IDs
        unique_model_ids = np.unique(self.model_ids)
        
        # Split model IDs into train, val, and test sets
        train_model_ids, temp_model_ids = train_test_split(
            unique_model_ids, test_size=(val_proportion + test_proportion), random_state=random_state
        )
        val_model_ids, test_model_ids = train_test_split(
            temp_model_ids, test_size=test_proportion / (val_proportion + test_proportion), random_state=random_state
        )
        
        # Now, filter the dataset's data points based on the model IDs
        train_indices = [i for i, model_id in enumerate(self.model_ids) if model_id in train_model_ids]
        val_indices = [i for i, model_id in enumerate(self.model_ids) if model_id in val_model_ids]
        test_indices = [i for i, model_id in enumerate(self.model_ids) if model_id in test_model_ids]
        
        # Create subsets
        train_dataset = Subset(self, train_indices)
        val_dataset = Subset(self, val_indices)
        test_dataset = Subset(self, test_indices)
        
        # DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        # Return the DataLoaders
        return train_loader, val_loader, test_loader

# Models


## UNetLSTM


In [3]:
class UNetLSTM(nn.Module):
    def __init__(self, in_channels=3, out_channels=2, lstm_hidden_size=512, lstm_num_layers=2):
        super(UNetLSTM, self).__init__()
        
        # Encoding Part
        self.encoder = nn.Sequential(
            self._conv_block(in_channels, 64),
            nn.MaxPool2d(2),
            self._conv_block(64, 128),
            nn.MaxPool2d(2),
            self._conv_block(128, 256),
            nn.MaxPool2d(2),
            self._conv_block(256, 512),
            nn.MaxPool2d(2),
            self._conv_block(512, 1024),
            nn.MaxPool2d(2)
        )
        
        # Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d(1)
        
        # LSTM Part
        self.lstm = nn.LSTM(1024, lstm_hidden_size, lstm_num_layers, batch_first=True)
        
        # Decoding Part
        self.decoder = nn.Sequential(
            self._upconv_block(lstm_hidden_size, 1024),
            self._conv_block(1024, 512),
            self._upconv_block(512, 512),
            self._conv_block(512, 256),
            self._upconv_block(256, 256),
            self._conv_block(256, 128),
            self._upconv_block(128, 128),
            self._conv_block(128, 64),
            self._upconv_block(64, 64),
            self._conv_block(64, 32),
            self._upconv_block(32, 32),
            self._conv_block(32, 16),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            self._conv_block(16, 16)
        )
        
        # Final Convolution
        self.output_conv = nn.Conv2d(16, out_channels, kernel_size=1)
        
        
    def _conv_block(self, in_channels, features):
        return nn.Sequential(
            nn.Conv2d(in_channels, features, 3, padding=1),
            nn.BatchNorm2d(features),
            nn.ReLU(),
            nn.Conv2d(features, features, 3, padding=1),
            nn.BatchNorm2d(features),
            nn.ReLU()
        )
    
    def _upconv_block(self, in_channels, features):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, features, kernel_size=2, stride=2),
            nn.BatchNorm2d(features),
            nn.ReLU()
        )

    def forward(self, x_seq):
        # x_seq shape: (batch_size, seq_length, in_channels, height, width)
        batch_size, seq_length, _, _, _ = x_seq.size()
        
        # Pass each state through the encoding part
        encoded_states = []
        for t in range(seq_length):
            x = x_seq[:, t, :, :, :]
            x = self.encoder(x)
            x = self.gap(x)
            x = x.view(batch_size, -1)  # Flatten the feature maps
            encoded_states.append(x)
        
        # Stack the encoded states
        encoded_states = torch.stack(encoded_states, dim=1)
        
        # Pass the encoded states through the LSTM
        lstm_output, _ = self.lstm(encoded_states)
        
        # Pass the LSTM output through the decoding part
        x = lstm_output[:, -1, :]  # Take only the last state
        x = x.view(batch_size, -1, 1, 1)  # Reshape to (batch_size, features, 1, 1)
        x = self.decoder(x)
        x = self.output_conv(x)
        
        return x

    # def forward(self, x_seq):
    #     # x_seq shape: (batch_size, seq_length, in_channels, height, width)
    #     batch_size, seq_length, _, _, _ = x_seq.size()
        
    #     # Pass each state through the encoding part
    #     encoded_states = []
    #     for t in range(seq_length):
    #         x = x_seq[:, t, :, :, :]
    #         x = self.encoder(x)
    #         x = self.gap(x)
    #         x = x.view(batch_size, -1)  # Flatten the feature maps
    #         encoded_states.append(x)
        
    #     # Stack the encoded states
    #     encoded_states = torch.stack(encoded_states, dim=1)
        
    #     # Pass the encoded states through the LSTM
    #     lstm_output, _ = self.lstm(encoded_states)
        
    #     # Pass the LSTM output through the decoding part
    #     decoded_states = []
    #     for t in range(seq_length):
    #         x = lstm_output[:, t, :]
    #         x = x.view(batch_size, -1, 1, 1)  # Reshape to (batch_size, features, 1, 1)
    #         x = self.decoder(x)
    #         x = self.output_conv(x)
    #         decoded_states.append(x)
        
    #     # Stack the decoded states
    #     decoded_states = torch.stack(decoded_states, dim=1)
        
    #     return decoded_states

# Trainer Class

In [4]:
class Trainer:
    def __init__(self, model, optimizer, criterion, device, model_name=""):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device
        self.model_name = model_name    
        self.training_losses = []
        self.validation_losses = []

    def train(self, train_loader, val_loader, epochs, checkpoint_interval=5):
        os.makedirs(f'{self.model_name}_checkpoints', exist_ok=True)

        self.model.train()
        for epoch in range(epochs):
            self.model.train()
            total_loss = 0.0
            for data in train_loader:
                current_seq, next_seq = data
                current_seq = current_seq.to(self.device)
                next_seq = next_seq.to(self.device)

                # Squeeze the sequence length dimension from next_seq
                next_seq = next_seq.squeeze(1)
                
                self.optimizer.zero_grad()
                predictions = self.model(current_seq)
                loss = self.criterion(predictions, next_seq)
                loss.backward()
                self.optimizer.step()
                
                total_loss += loss.item()
            
            avg_loss = total_loss / len(train_loader)
            self.training_losses.append(avg_loss)
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

            # Validation step
            val_loss = self.validate(val_loader)
            self.validation_losses.append(val_loss)

            # Save the model at the specified checkpoint interval
            if (epoch + 1) % checkpoint_interval == 0:
                self.save_checkpoint(epoch + 1)
                self.save_losses()

        # Save losses after the final epoch
        self.save_losses()

        # After training, plot the training and validation losses
        self.plot_losses()

    def validate(self, val_loader):
        self.model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for data in val_loader:
                current_seq, next_seq = data
                current_seq = current_seq.to(self.device)
                next_seq = next_seq.to(self.device)

                # Squeeze the sequence length dimension from next_seq
                next_seq = next_seq.squeeze(1)

                predictions = self.model(current_seq)
                loss = self.criterion(predictions, next_seq)
                total_val_loss += loss.item()
        avg_val_loss = total_val_loss / len(val_loader)
        print(f'Validation Loss: {avg_val_loss:.4f}')
        return avg_val_loss

    def save_checkpoint(self, epoch):
        checkpoint_path = f'model_checkpoints/model_epoch_{epoch}.pth'
        torch.save(self.model.state_dict(), checkpoint_path)
        print(f'Model saved to {checkpoint_path}')

    def save_losses(self):
        losses = {
            'training_losses': self.training_losses,
            'validation_losses': self.validation_losses
        }
        with open(f'{self.model_name}_losses.json', 'w') as f:
            json.dump(losses, f)
        print(f'Losses saved to {self.model_name}_losses.json')

    def load_losses(self):
        with open(f'{self.model_name}_losses.json', 'r') as f:
            losses = json.load(f)
        self.training_losses = losses['training_losses']
        self.validation_losses = losses['validation_losses']
        print(f'Losses loaded from {self.model_name}_losses.json')

# Model Set Up

In [5]:
# Data
root_dir = 'C:\\Users\\thomas.bush\\repos\\dyna-landslide-surrogate\\data_small'

batch_size = 16

split_proportions = (0.7, 0.15, 0.15)

epochs = 5

series_length = 3

In [6]:
# Initialize dataset with scaling
dataset = Debris_State_Series_Dataset(root_dir, array_size=128, series_length=series_length, apply_scaling=True)

# Split dataset into train, validation, and test sets and create dataloaders
train_loader, val_loader, test_loader = dataset.create_dataloaders(split_proportions, batch_size, random_state=42)

# Compute scaling factors based on the train dataset, but only if scaling is applied
if dataset.apply_scaling:
    train_dataset = train_loader.dataset.dataset 
    dataset.compute_scaling_factors()

# Dataset stats (optional)
print(f"Total dataset size: {len(dataset)}")
print(f"Train size: {len(train_loader.dataset)}, Validation size: {len(val_loader.dataset)}, Test size: {len(test_loader.dataset)}")

Total dataset size: 2265
Train size: 1430, Validation size: 481, Test size: 354


In [7]:
# Get the first batch of data
first_batch = next(iter(train_loader))

# Print the first batch
print(first_batch)


[tensor([[[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.000

In [8]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on device: {device}.")

Training on device: cuda.


In [9]:
# List of models to train
# models = [
#     {'model': Complex_CNN(), 'name': 'complex_cnn'},
#     {'model': UNet(), 'name': 'unet'},
#     {'model': LargeUNet(), 'name': 'large_unet'}
# ]

# models = [
#     {'model': Complex_CNN(), 'name': 'complex_cnn'}
# ]

models = [
    {'model': UNetLSTM(in_channels=3, out_channels=2, lstm_hidden_size=512, lstm_num_layers=2), 'name': 'unetlstm'}
]

# models = [
#     {'model': LargeUNet(), 'name': 'large_unet'}
# ]



In [10]:
dataset.data_series

[(['C:\\Users\\thomas.bush\\repos\\dyna-landslide-surrogate\\data_small\\00002\\04_FinalProcessedData_128\\velocity\\00002_velocity_2.npy',
   'C:\\Users\\thomas.bush\\repos\\dyna-landslide-surrogate\\data_small\\00002\\04_FinalProcessedData_128\\velocity\\00002_velocity_3.npy',
   'C:\\Users\\thomas.bush\\repos\\dyna-landslide-surrogate\\data_small\\00002\\04_FinalProcessedData_128\\velocity\\00002_velocity_4.npy',
   'C:\\Users\\thomas.bush\\repos\\dyna-landslide-surrogate\\data_small\\00002\\04_FinalProcessedData_128\\velocity\\00002_velocity_5.npy'],
  ['C:\\Users\\thomas.bush\\repos\\dyna-landslide-surrogate\\data_small\\00002\\04_FinalProcessedData_128\\thickness\\00002_thickness_2.npy',
   'C:\\Users\\thomas.bush\\repos\\dyna-landslide-surrogate\\data_small\\00002\\04_FinalProcessedData_128\\thickness\\00002_thickness_3.npy',
   'C:\\Users\\thomas.bush\\repos\\dyna-landslide-surrogate\\data_small\\00002\\04_FinalProcessedData_128\\thickness\\00002_thickness_4.npy',
   'C:\\Users

In [11]:

# Train each model
for model_info in models:
    # Move model to the appropriate device
    model = model_info['model'].to(device)
    
    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = Adam(model.parameters(), lr=1e-3)
    
    # Initialize the trainer
    model_name = model_info['name']
    trainer = Trainer(model, optimizer, criterion, device, model_name=model_name + "_test")
    
    # Train the model
    print(f"Training {model_name}...")
    trainer.train(train_loader, val_loader, epochs=epochs, checkpoint_interval=5)
    
    # Save the trained model state
    torch.save(model.state_dict(), f"{model_name}_state.pth")

    print(f"Finished training {model_name}. Model state saved.")

Training unetlstm...
Epoch [1/5], Loss: 0.0659
Validation Loss: 0.1383
Epoch [2/5], Loss: 0.0583
Validation Loss: 0.1386
Epoch [3/5], Loss: 0.0576
Validation Loss: 0.1348
Epoch [4/5], Loss: 0.0566
Validation Loss: 0.1390
Epoch [5/5], Loss: 0.0553
Validation Loss: 0.1343
Model saved to model_checkpoints/model_epoch_5.pth
Losses saved to unetlstm_test_losses.json
Losses saved to unetlstm_test_losses.json


AttributeError: 'Trainer' object has no attribute 'plot_losses'

In [ ]:
trainer.plot_losses()

In [12]:
trainer.test(test_loader)



AttributeError: 'Trainer' object has no attribute 'test'

In [ ]:
trainer.plot_predictions(test_loader, num_predictions=1)